In [1]:
import os
import glob
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
def readBinaryData(n,SIZE,H,nbytes,BO='BE'):

    if nbytes==2:
        d = np.zeros((SIZE,SIZE,H),np.uint16)
    elif nbytes==1:
        d = np.zeros((SIZE,SIZE,H),np.uint8)
    else:
        print('Wrong number of bytes per voxel')
        return
    
    f=open(n,"rb")
    for i in range(0,H):
        for j in range(0,SIZE):
            for k in range(0,SIZE):
                byte = f.read(nbytes)
                if nbytes==2:
                    if BO =='BE':
                        a = 256*byte[0] + byte[1]
                    elif BO == 'LE':
                        a = byte[0] + 256*byte[1]
                        
                else:
                    a = byte[0]
                d[j,k,i] = a
    f.close()
    return d


## Configuration

In [3]:
LEFT_BONE_TOP = 60
LEFT_BONE_BOTTOM = 70
RIGHT_BONE_TOP = 80
RIGHT_BONE_BOTTOM = 90
LEFT_MIDDLE_BONE_TOP = 100
LEFT_MIDDLE_BONE_BOTTOM = 110
RIGHT_MIDDLE_BONE_TOP = 120
RIGHT_MIDDLE_BONE_BOTTOM = 130

REFERENCE = 50
JOINT_LINE = 200

## Create lists for evaluation

In [6]:
REF_REG_DIR = '/home/user/Spine/Data/Regions/'
#REF_LES_DIR = '/home/user/Spine/Data/GroundTruth_MajorityVoting/'
REF_LES_DIR = '/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/'


EVAL_REG_DIR = '/home/user/Spine/Wyniki_v1/Segmentacja/MyUnet/AfterStep6_PredRegions/'
EVAL_LES_DIR = '/home/user/Spine/Wyniki_v1/Klasyfikacja/ScisleRozdzielenieTestowOdTreningu/'

#EVAL_REG_DIR = '/home/user/Spine/Wyniki_v0/Segmentacja/PredictedBoneLabels/PredictedRegions/'
#EVAL_LES_DIR = '/home/user/Spine/Wyniki_v0/Klasyfikacja/Iteration2/'

RefRegNameMask = 'Regions*.raw'
EvalRegNameMask = 'Regions*.raw'

RefLesNameMask = 'VerMV*.raw'
EvalLesNameMask = 'Recognitions*.raw'

RefLesionsList = glob.glob(REF_LES_DIR + RefLesNameMask)
RefLesionsList.sort()
numbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in RefLesionsList]
print(numbers)

RefRegionsList = [f for f in glob.glob(REF_REG_DIR+RefRegNameMask) if f.split('/')[-1].split('.')[0].split('_')[1] in numbers]
RefRegionsList.sort()
RefRegionsNumbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in RefRegionsList]
print(RefRegionsNumbers)

EvalLesionsList = [f for f in glob.glob(EVAL_LES_DIR+EvalLesNameMask) if f.split('/')[-1].split('.')[0].split('_')[1] in numbers]
EvalLesionsList.sort()
EvalLesionsNumbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in EvalLesionsList]
print(EvalLesionsNumbers)

EvalRegionsList = [f for f in glob.glob(EVAL_REG_DIR+EvalRegNameMask) if f.split('/')[-1].split('.')[0].split('_')[1] in numbers]
EvalRegionsList.sort()
EvalRegionsNumbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in EvalRegionsList]
print(EvalRegionsNumbers)


['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']
['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']
['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']
['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']


## Pixel-wise evaluation

In [ ]:


SIZE = 400

TP = 0
TN = 0
FP = 0
FN = 0
for lesRef,lesEval,regRef,regEval in zip(RefLesionsList,EvalLesionsList,RefRegionsList,EvalRegionsList):
    H1 = int(lesRef.split('/')[-1].split('.')[0].split('_')[4])
    H2 = int(lesEval.split('/')[-1].split('.')[0].split('_')[4])
    H3 = int(regRef.split('/')[-1].split('.')[0].split('_')[4])
    H4 = int(regEval.split('/')[-1].split('.')[0].split('_')[4])
    H = min(H1,H2,H3,H4)
#    print(lesRef,H1,H2,H3,H4)
    lesR = readBinaryData(lesRef,SIZE,H1,1)[:,:,:H]
    lesE = readBinaryData(lesEval,SIZE,H2,1)[:,:,:H]
    regR = readBinaryData(regRef,SIZE,H3,1)[:,:,:H]
    regE = readBinaryData(regEval,SIZE,H4,1)[:,:,:H]
    lesR[lesR!=0] = 255
    lesE[lesE!=0] = 255
    lesR[regR<LEFT_BONE_TOP] = 128
    lesR[regR>RIGHT_MIDDLE_BONE_BOTTOM] = 128
    lesE[regE<LEFT_BONE_TOP] = 128
    lesE[regE>RIGHT_MIDDLE_BONE_BOTTOM] = 128
    
    tp = 0
    tn = 0 
    fp = 0
    fn = 0
    for R in range(SIZE):
        for C in range(SIZE):
            for S in range(H):
                if lesE[R,C,S]==255 and lesR[R,C,S]==255:
                    tp += 1
                if lesE[R,C,S]==0 and lesR[R,C,S]==0:
                    tn +=1
                if lesE[R,C,S]==255 and lesR[R,C,S]==0:
                    fp +=1
                if lesE[R,C,S]==0 and lesR[R,C,S]==255:
                    fn +=1
                    
    print(lesRef,tp,tn,fp,fn)

    TP += tp
    TN += tn
    FP += fp
    FN += fn
    #break

print(TP,TN,FP,FN,TP/(TP+FN),TN/(TN+FP))     #TP,TN,FP,FN,SENSITIVITY,SPECIFICITY
    

/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/VerMV_10_400_400_18_1_.raw 193 55456 305 307
/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/VerMV_11_400_400_18_1_.raw 2028 57149 1372 482
/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/VerMV_1_400_400_18_1_.raw 26034 40639 1026 1079
/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/VerMV_24_400_400_20_1_.raw 21 57970 5235 5
/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/VerMV_25_400_400_21_1_.raw 0 52880 390 0
/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/VerMV_26_400_400_18_1_.raw 875 64699 2245 131
/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/VerMV_27_400_400_20_1_.raw 774 37077 1660 1376
/home/user/Spine/Wyniki_v0/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/VerMV_28_400_400_18_1_.raw 1874 47057 2132 

## Quadrant-wise evaluation

In [ ]:


SIZE = 400

TP = 0
TN = 0
FP = 0
FN = 0
for lesRef,lesEval,regRef,regEval in zip(RefLesionsList,EvalLesionsList,RefRegionsList,EvalRegionsList):
    H1 = int(lesRef.split('/')[-1].split('.')[0].split('_')[4])
    H2 = int(lesEval.split('/')[-1].split('.')[0].split('_')[4])
    H3 = int(regRef.split('/')[-1].split('.')[0].split('_')[4])
    H4 = int(regEval.split('/')[-1].split('.')[0].split('_')[4])
    H = min(H1,H2,H3,H4)
    lesR = readBinaryData(lesRef,SIZE,H1,1)[:,:,:H]
    lesE = readBinaryData(lesEval,SIZE,H2,1)[:,:,:H]
    regR = readBinaryData(regRef,SIZE,H3,1)[:,:,:H]
    regE = readBinaryData(regEval,SIZE,H4,1)[:,:,:H]
    lesR[lesR!=0] = 255
    lesE[lesE!=0] = 255
    lesR[regR<LEFT_BONE_TOP] = 128
    lesR[regR>RIGHT_MIDDLE_BONE_BOTTOM] = 128
    lesE[regE<LEFT_BONE_TOP] = 128
    lesE[regE>RIGHT_MIDDLE_BONE_BOTTOM] = 128
    
    quadRef = np.zeros((8,),dtype=np.int8)
    quadRef.fill(-1)
    for R in range(SIZE):
        for C in range(SIZE):
            for S in range(H):
                if regR[R,C,S] >= LEFT_BONE_TOP and regR[R,C,S]<=RIGHT_MIDDLE_BONE_BOTTOM:
                    pos = (regR[R,C,S] - LEFT_BONE_TOP)//10
                    if quadRef[pos]==-1:
                        quadRef[pos] = 0
                    if lesR[R,C,S]:
                        quadRef[pos] = 1

    quadEval = np.zeros((8,),dtype=np.int8)
    quadEval.fill(-1)
    for R in range(SIZE):
        for C in range(SIZE):
            for S in range(H):
                if regE[R,C,S] >= LEFT_BONE_TOP and regE[R,C,S]<=RIGHT_MIDDLE_BONE_BOTTOM:
                    pos = (regE[R,C,S] - LEFT_BONE_TOP)//10
                    if quadEval[pos]==-1:
                        quadEval[pos] = 0
                    if lesE[R,C,S]:
                        quadEval[pos] = 1
    
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for pos in range(8):
        if quadRef[pos]>=0:
            if quadRef[pos]==1 and quadEval[pos]==1:
                tp += 1
            if quadRef[pos]==1 and quadEval[pos]!=1:
                fn += 1
            if quadRef[pos]!=1 and quadEval[pos]==1:
                fp += 1
            if quadRef[pos]!=1 and quadEval[pos]!=1:
                tn += 1
        
    print(lesRef,tp,tn,fp,fn)

    TP += tp
    TN += tn
    FP += fp
    FN += fn
    #break

print(TP,TN,FP,FN,TP/(TP+FN),TN/(TN+FP))     #TP,TN,FP,FN,SENSITIVITY,SPECIFICITY
 

## Create file lists for verification

In [8]:
STIR_DIR = '/home/user/Spine/Data/STIR/'
RECOGN_DIR = '/home/user/Spine/Wyniki/Klasyfikacja/Iteration2/'
REF_LES_DIR = '/home/user/Spine/Data/GroundTruth_MajorityVoting/'
REF_REG_DIR = '/home/user/Spine/Data/Regions/'

RecognList = glob.glob(RECOGN_DIR + 'RecognitionsOR*.raw')
RecognList.sort()
numbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in RecognList]
print(numbers)

StirList = [f for f in glob.glob(STIR_DIR+'STIR*.raw') if f.split('/')[-1].split('.')[0].split('_')[1] in numbers]
StirList.sort()
StirNumbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in StirList]
print(StirNumbers)

MajorVotList = [f for f in glob.glob(REF_LES_DIR + 'MV*.raw')  if f.split('/')[-1].split('.')[0].split('_')[1] in numbers]
MajorVotList.sort()
MajorVotNumbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in MajorVotList]
print(MajorVotNumbers)

RefRegionsList = [f for f in glob.glob(REF_REG_DIR+'Regions*.raw') if f.split('/')[-1].split('.')[0].split('_')[1] in numbers]
RefRegionsList.sort()
RefRegionsNumbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in RefRegionsList]
print(RefRegionsNumbers)


['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']
['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']
['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']
['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']


## Create files for verification

In [14]:

def writeBinaryData(d,n):

    f=open(n,"wb")
    for i in range(0,d.shape[2]):
        for j in range(0,d.shape[0]):
            for k in range(0,d.shape[0]):
                byte = f.write(d[j,k,i])
    f.close()

SIZE = 400
for l,s,m,r in zip(RefLesionsList,StirList,MajorVotList,RefRegionsList):
    H1 = int(l.split('/')[-1].split('.')[0].split('_')[4])
    H2 = int(s.split('/')[-1].split('.')[0].split('_')[4])
    H3 = int(m.split('/')[-1].split('.')[0].split('_')[4])
    H4 = int(r.split('/')[-1].split('.')[0].split('_')[4])
    H = min(H1,H2,H3,H4)
    
    les = readBinaryData(l,SIZE,H1,1)[:,:,:H]
    stir = readBinaryData(s,SIZE,H2,2)[:,:,:H]
    mv = readBinaryData(m,SIZE,H1,1)[:,:,:H]
    region = readBinaryData(r,SIZE,H1,1)[:,:,:H]
    
    fp = np.zeros((SIZE,SIZE,H),dtype=np.uint8)
    fn = np.zeros((SIZE,SIZE,H),dtype=np.uint8)

    suma = np.zeros((SIZE,SIZE,H),dtype=np.uint16)
    sumafp = np.zeros((SIZE,SIZE,H),dtype=np.uint16)
    sumafn = np.zeros((SIZE,SIZE,H),dtype=np.uint16)

    m = np.max(stir)
    print(m,les.shape,stir.shape,np.max(les))

    np.copyto(suma,stir)
    suma[les==255] = m
    name = 'Sum_' + l.split('/')[-1]
    writeBinaryData(suma,name)
    
    np.copyto(fp,les)
    fp[mv!=0] = 0
    fp[region<LEFT_BONE_TOP] = 0
    fp[region>RIGHT_MIDDLE_BONE_BOTTOM] = 0
    np.copyto(sumafp,stir)
    sumafp[fp==255] = m
    name = 'SumFP_' + l.split('/')[-1]
    writeBinaryData(sumafp,name)
    name = 'FP_' + l.split('/')[-1]
    writeBinaryData(fp,name)
    
    np.copyto(fn,mv)
    fn[les!=0] = 0
    fn[region<LEFT_BONE_TOP] = 0
    fn[region>RIGHT_MIDDLE_BONE_BOTTOM] = 0
    np.copyto(sumafn,stir)
    sumafn[fn==255] = m
    name = 'SumFN_' + l.split('/')[-1]
    writeBinaryData(sumafn,name)
    name = 'FN_' + l.split('/')[-1]
    writeBinaryData(fn,name)
    
    #break


1638 (400, 400, 18) (400, 400, 18) 255
1523 (400, 400, 18) (400, 400, 18) 255
1360 (400, 400, 18) (400, 400, 18) 255
1548 (400, 400, 20) (400, 400, 20) 255
1474 (400, 400, 21) (400, 400, 21) 255
1364 (400, 400, 18) (400, 400, 18) 255
1531 (400, 400, 20) (400, 400, 20) 255
1145 (400, 400, 18) (400, 400, 18) 255
1380 (400, 400, 18) (400, 400, 18) 255
1478 (400, 400, 18) (400, 400, 18) 255
1638 (400, 400, 22) (400, 400, 22) 255
2500 (400, 400, 24) (400, 400, 24) 255
2759 (400, 400, 18) (400, 400, 18) 255
2891 (400, 400, 18) (400, 400, 18) 255
2458 (400, 400, 18) (400, 400, 18) 255
2613 (400, 400, 20) (400, 400, 20) 255
2509 (400, 400, 18) (400, 400, 18) 255
2377 (400, 400, 21) (400, 400, 21) 255
2360 (400, 400, 18) (400, 400, 18) 255
2541 (400, 400, 22) (400, 400, 22) 255
849 (400, 400, 18) (400, 400, 18) 255
2323 (400, 400, 18) (400, 400, 18) 255
2738 (400, 400, 18) (400, 400, 18) 255
1191 (400, 400, 22) (400, 400, 22) 255
557 (400, 400, 18) (400, 400, 18) 255
440 (400, 400, 18) (400, 40

## Combine verified files with original recognitions to get final groung-truth

In [9]:
VERIF_DIR = '/home/user/Spine/Wyniki/Klasyfikacja/EwaluacjaIteracji2/WeryfikacjaIteracji2/'
REF_LES_DIR = '/home/user/Spine/Data/GroundTruth_MajorityVoting/'

MajorVotList = [f for f in glob.glob(REF_LES_DIR + 'MV*.raw')]
MajorVotList.sort()
MajorVotNumbers = [l.split('/')[-1].split('.')[0].split('_')[1] for l in MajorVotList]
print(MajorVotNumbers)

VerifList = [f for f in glob.glob(VERIF_DIR+'FP*.raw') if f.split('/')[-1].split('.')[0].split('_')[2] in MajorVotNumbers]
VerifList.sort()
VerifNumbers = [l.split('/')[-1].split('.')[0].split('_')[2] for l in VerifList]
print(VerifNumbers)


['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']
['10', '11', '1', '24', '25', '26', '27', '28', '29', '2', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '3', '40', '41', '42', '4', '5', '6', '7', '8', '9']


In [13]:
def writeBinaryData(d,n):

    f=open(n,"wb")
    for i in range(0,d.shape[2]):
        for j in range(0,d.shape[0]):
            for k in range(0,d.shape[0]):
                byte = f.write(d[j,k,i])
    f.close()

SIZE = 400
for m,r in zip(MajorVotList,VerifList):
    H1 = int(m.split('/')[-1].split('.')[0].split('_')[4])
    H2 = int(r.split('/')[-1].split('.')[0].split('_')[5])
    H = min(H1,H2)
    
    major = readBinaryData(m,SIZE,H1,1)[:,:,:H]
    ver = readBinaryData(r,SIZE,H2,1)[:,:,:H]
    
    major[ver==255] = 255
    
    name = VERIF_DIR + 'Ver' + m.split('/')[-1]
    writeBinaryData(major,name)
    
    #break
